#### Let's Begin by importing both the libs sklearn and cuml to measure the performance difference

Starting with the basics lets try Random Forest Classifier for checking heardisease

In [ ]:
!nvidia-smi

In [ ]:
import cudf as df
import pandas as pd

## Let's import the data now

In [ ]:
data_df=df.read_csv("heart-disease.csv")
data_pd=pd.read_csv("heart-disease.csv")

In [ ]:
data_df

### getting training and test data split ready 

In [ ]:
from sklearn.model_selection import train_test_split as tts_sk
from cuml.model_selection import train_test_split as tts_ml

In [ ]:
x=data_df.drop('target',axis=1)
y=data_df['target']

X=data_pd.drop('target',axis=1)
Y=data_pd['target']

In [ ]:
x_train,x_test,y_train,y_test = tts_ml(x,y,test_size=0.2)

In [ ]:
X_train,X_test,Y_train,Y_test = tts_sk(X,Y,test_size=0.2)

### Importing Models 

In [ ]:
from cuml.ensemble import RandomForestClassifier as rfc_ml
from sklearn.ensemble import RandomForestClassifier as rfc_sk

### Fittin Data

In [ ]:
clf_ml=rfc_ml()
clf_sk=rfc_sk()

In [ ]:
from cuml.model_selection import GridSearchCV as GridSearchCV_ml
from sklearn.model_selection import GridSearchCV as GridSearchCV_sk


import cupy as py 
import numpy as np

### Trying Models CUML
 

In [ ]:
clf_ml.fit(x_train,y_train)

In [ ]:
clf_ml.score(x_test,y_test)

### Trying Models Sklearn


In [ ]:
clf_sk.fit(X_train,Y_train)

In [ ]:
clf_sk.score(X_test,Y_test)

In [ ]:
param_grid = {'n_estimators': [i for i in range(10, 100, 1)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_ml,
                    param_grid,
                    cv=5,
                    verbose=True)

# Fit the grid search to the data
grid.fit(X_train, Y_train)

# Find the best parameters
grid.best_params_

In [ ]:
param_grid = {'n_estimators': [i for i in range(10, 100, 1)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_sk,
                    param_grid,
                    cv=5,
                    verbose=True,
                    n_jobs=-1)

# Fit the grid search to the data
grid.fit(X_train, Y_train)

# Find the best parameters
grid.best_params_

## Time to Test Bigger Data

### Bulldoser Data 

Note: this notebook from further on will only contain CuML code. If you're intersted in Sklearn look at the the bulldoser.ipynb in the same repo

In [106]:
import cudf as df
from cuml.model_selection import train_test_split
import cupy as py 

In [123]:
dt=df.read_csv("bluebook-for-bulldozers/TrainAndValid.csv",parse_dates=['saledate'])

In [125]:
dt

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3,2004,68,Low,2006-11-16,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
1,1139248,57000.0,117657,77,121,3,1996,4640,Low,2004-03-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3,2001,2838,High,2004-02-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,1139251,38500.0,1026470,332,121,3,2001,3486,High,2011-05-19,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1139253,11000.0,1057373,17311,121,3,2007,722,Medium,2009-07-23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412693,6333344,10000.0,1919201,21435,149,2,2005,<NA>,<NA>,2012-03-07,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412694,6333345,10500.0,1882122,21436,149,2,2005,<NA>,<NA>,2012-01-28,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412695,6333347,12500.0,1944213,21435,149,2,2005,<NA>,<NA>,2012-01-28,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412696,6333348,10000.0,1794518,21435,149,2,2006,<NA>,<NA>,2012-03-07,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>


In [109]:
dt.isnull().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [110]:
len(dt)

412698

In [111]:
dt.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   SalesID                   412698 non-null  int64
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64
 3   ModelID                   412698 non-null  int64
 4   datasource                412698 non-null  int64
 5   auctioneerID              392562 non-null  int64
 6   YearMade                  412698 non-null  int64
 7   MachineHoursCurrentMeter  147504 non-null  int64
 8   UsageBand                 73670 non-null   object
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object
 11  fiBaseModel               412698 non-null  object
 12  fiSecondaryDesc           271971 non-null  object
 13  fiModelSeries             58667 non-null   object
 14  fi

In [112]:
dt=dt.sort_values(by="saledate")

In [113]:
dt

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500.0,1126363,8434,132,18,1974,<NA>,<NA>,1989-01-17,...,<NA>,<NA>,<NA>,<NA>,<NA>,None or Unspecified,Straight,None or Unspecified,<NA>,<NA>
28603,1259551,9500.0,1292226,6788,132,18,1977,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28757,1259760,8000.0,1476205,6788,132,99,1978,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28820,1259842,9000.0,1226323,6788,132,18,1978,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
28927,1259973,7700.0,1150485,6788,132,18,1978,<NA>,<NA>,1989-01-31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412164,6318085,5500.0,1873607,10402,149,99,2001,<NA>,<NA>,2012-04-28,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
412218,6319596,20500.0,1887135,18110,149,99,2005,<NA>,<NA>,2012-04-28,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
412436,6321656,14000.0,1823084,18631,149,99,2006,<NA>,<NA>,2012-04-28,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
412476,6324811,6000.0,1919104,19330,149,99,2004,<NA>,<NA>,2012-04-28,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [114]:
dt["saleDay"]=dt.saledate.dt.day
dt["saleMonth"]=dt.saledate.dt.month
dt["saleYear"]=dt.saledate.dt.year

dt["saleDayofweek"] = dt.saledate.dt.dayofweek
dt["saleDayofyear"] = dt.saledate.dt.dayofyear

In [115]:
dt=dt.drop("saledate",axis=1)

In [121]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=data.astype('category').cat.as_ordered()

TypeError: CategoricalAccessor.as_ordered() got an unexpected keyword argument 'nan_as_null'

### The Above code is producing error as the size is diffrent. Check thread
 https://stackoverflow.com/questions/42382263/valueerror-length-of-values-does-not-match-length-of-index-pandas-dataframe-u

### Initial Solution is too fill all the nan values with something 

### Lets Try that 

In [ ]:
# dic=dt.items()

# for column,data in dic:
#     if pd.api.types.is_object_dtype(data):
#         dt[column].fillna("Empty")

In [117]:
dt.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 412698 entries, 205615 to 412544
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   SalesID                   412698 non-null  int64
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64
 3   ModelID                   412698 non-null  int64
 4   datasource                412698 non-null  int64
 5   auctioneerID              392562 non-null  int64
 6   YearMade                  412698 non-null  int64
 7   MachineHoursCurrentMeter  147504 non-null  int64
 8   UsageBand                 73670 non-null   category
 9   fiModelDesc               412698 non-null  object
 10  fiBaseModel               412698 non-null  object
 11  fiSecondaryDesc           271971 non-null  object
 12  fiModelSeries             58667 non-null   object
 13  fiModelDescriptor         74816 non-null   object
 14  Pro

In [118]:
dt.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension             386715
Blade_Width         

In [119]:
dt.UsageBand.count()

73670

In [131]:
dt["UsageBand"]=dt.UsageBand.fillna(py.nan)

In [132]:
dt.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                        0
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [136]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=dt[column].fillna(py.nan)

In [137]:
dt.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                        0
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc                  0
fiModelSeries                    0
fiModelDescriptor                0
ProductSize                      0
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                     0
Enclosure                        0
Forks                            0
Pad_Type                         0
Ride_Control                     0
Stick                            0
Transmission                     0
Turbocharged                     0
Blade_Extension     

In [143]:
dt["auctioneerID"]=dt["auctioneerID"].fillna(-1)

In [146]:
dt["MachineHoursCurrentMeter"]=dt["MachineHoursCurrentMeter"].fillna(-1)

In [147]:
dt.isna().sum()

SalesID                     0
SalePrice                   0
MachineID                   0
ModelID                     0
datasource                  0
auctioneerID                0
YearMade                    0
MachineHoursCurrentMeter    0
UsageBand                   0
saledate                    0
fiModelDesc                 0
fiBaseModel                 0
fiSecondaryDesc             0
fiModelSeries               0
fiModelDescriptor           0
ProductSize                 0
fiProductClassDesc          0
state                       0
ProductGroup                0
ProductGroupDesc            0
Drive_System                0
Enclosure                   0
Forks                       0
Pad_Type                    0
Ride_Control                0
Stick                       0
Transmission                0
Turbocharged                0
Blade_Extension             0
Blade_Width                 0
Enclosure_Type              0
Engine_Horsepower           0
Hydraulics                  0
Pushblock 

In [148]:
dic=dt.items()

for column,data in dic:
    if pd.api.types.is_object_dtype(data):
        dt[column]=data.astype('category').cat.as_ordered()

ValueError: Length of values (206349) does not match length of index (412698)